# Test de comparaisons

In [1]:
# -*- coding: utf8 -*-
import codecs,glob,re,pickle,features,time
import pandas as pd
import itertools as it

In [2]:
import math
def rAn(r,n):
    f = math.factorial
    return f(n) / f(n-r)
def rCn(r,n):
    f = math.factorial
    return f(n) / f(r) / f(n-r)

In [21]:
features.add_config('../bdlexique.ini')
fs=features.FeatureSystem('phonemes')
phonologicalMap="-X"

In [22]:
filePrefix="/Users/gilles/Box Sync/2015-Data/FlexionAdjectifs/"

In [23]:
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [24]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [7]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

In [8]:
def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

In [371]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*" or morceau==".*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

In [29]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

In [18]:
class formesPatron:
    '''
    Accumulateur de formes correspondant à un patron pour calcul de la Généralisation Minimale (cf. MGL)
    '''
    def __init__(self):
        self.formes=[]

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterForme(self,forme):
        self.formes.append(forme)
        
    def calculerGM(self):
        minLongueur=len(min(self.formes, key=len))
        maxLongueur=len(max(self.formes, key=len))
        if debug: print (minLongueur, maxLongueur)
        positions=[]
        if maxLongueur>minLongueur:
            positions.append("*")
        for i in xrange(minLongueur, 0, -1):
            phonemes=set([x[-i] for x in self.formes])
            if debug1: print "phonemes","".join(phonemes)
            if "." in phonemes:
                positions.append(".")
            else:
                positions.append(u"".join(fs.lattice[phonemes].extent))
        return patron2regexp(positions)

class pairePatrons:
    '''
    Accumulateur de triplets (f1,f2,patron) correspondant à une paire pour calcul des Généralisations Minimales (cf. MGL)
    '''
    def __init__(self,case1,case2):
        self.patrons1={}
        self.patrons2={}
        self.case1=case1
        self.case2=case2

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterFormes(self,forme1,forme2,patron):
#        print (forme1,forme2,patron, file=logfile)
        patron12=patron
        (pat1,pat2)=patron.split("-")
        patron21=pat2+"-"+pat1
#        print (patron12,patron21, file=logfile)
        if not patron12 in self.patrons1:
            if debug: print (forme1,forme2,patron12)
            self.patrons1[patron12]=formesPatron()
        self.patrons1[patron12].ajouterForme(forme1)
        if not patron21 in self.patrons2:
            self.patrons2[patron21]=formesPatron()
        self.patrons2[patron21].ajouterForme(forme2)
        
        
    def calculerGM(self):
        resultat1={}
        for patron in self.patrons1:
            if debug: print ("patron1", patron)
            resultat1[patron]=self.patrons1[patron].calculerGM()
        resultat2={}
        for patron in self.patrons2:
            if debug: print ("patron2", patron)
            resultat2[patron]=self.patrons2[patron].calculerGM()
        return (resultat1,resultat2) 

In [11]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print ("le numéro de forme n'est pas dans [1,2]",n)# file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : calcule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        forme=recoder(forme)
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                print (forme)#, file=logfile)
                print ("pas de classe",idClasseForme)#, file=logfile)
                print ("%.2f par forme de sortie" % (float(1)/len(classeForme)))#, file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            print (forme)#, file=logfile) 
            print ("pas de patron")#, file=logfile)
        return sortieForme
        

In [25]:
def lireLexique(nomLexique):
    with open(nomLexique, 'rb') as input:
        lexique=pickle.load(input)
        lexique["phono"]=lexique["phono"].apply(recoder)
    return lexique

In [13]:
def lexique2Paradigmes(lexique):
    return pd.pivot_table(lexique, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()

In [14]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

In [26]:
listeEchantillons=glob.glob(filePrefix+"Longitudinal-*MSP.pkl")
#listeEchantillons

In [27]:
nomEchantillon=listeEchantillons[0]
lexique=lireLexique(nomEchantillon)
sampleCases=lexique.case.unique().tolist()
print (sampleCases)
paradigmes=lexique2Paradigmes(lexique)

[u'ms', u'mp', u'fs', u'fp']


In [31]:
debug=0
debug1=1
paradigme=paradigmes[["ms","fs","lexeme"]].dropna(thresh=3, axis=0).reindex()
(r1,r2)=rapports(paradigme)

phonemes p
phonemes r
phonemes O
phonemes t
phonemes E
phonemes k
phonemes t
phonemes 9
phonemes r
phonemes bf
phonemes rl
phonemes â
phonemes agfmlnSruw
phonemes aâEorô
phonemes l
phonemes ô
phonemes f
phonemes u
phonemes s
phonemes u
phonemes 9EHOSZabdgikjmlnpsrtwvyz
phonemes eikjlrtyE
phonemes bO
phonemes rm
phonemes û
phonemes Z
phonemes â
phonemes t
phonemes i
phonemes aâEiêoruô
phonemes sj
phonemes o
phonemes r
phonemes 6
phonemes m
phonemes i
phonemes sb
phonemes o
phonemes rbO
phonemes ruô
phonemes dt
phonemes 9
phonemes r
phonemes s
phonemes E
phonemes k
phonemes gfmlnrtvz
phonemes ê
phonemes bEdkjmSlsrtv
phonemes ê
phonemes aHnsrtvz
phonemes i9Eo
phonemes f
phonemes r
phonemes O
phonemes m
phonemes â
phonemes m
phonemes E
phonemes t
phonemes r
phonemes âEilOru6y
phonemes âEimlnsrt
phonemes Zj
phonemes e
phonemes pkbJf
phonemes ô
phonemes agfmlnSruw
phonemes aâEorô
phonemes d
phonemes bEdkjmSlsrtv
phonemes E
phonemes n
phonemes Z
phonemes â
phonemes t
phonemes i
phonemes j
pho

In [183]:
r1.patrons

{'-': u'^(.*[ptkbdgfsSvzZmnJNjlrwHiyEe926auOo\xea\xfb\xe2\xf4][ptkbdgfsSvzZjlrwHiyEe926uOo])$',
 u'-Es': u'^mEtr$',
 u'-S': u'^([pbfv][lr])\xe2$',
 u'-d': u'^(.*[ptkbdgfsSvzZmnJNjlrwHiyEe926auOo\xea\xfb\xe2\xf4][rEe926aOo\xea\xfb\xe2\xf4])$',
 u'-g': u'^l\xf4$',
 u'-j': u'^Z\xe2ti$',
 u'-l': u'^su$',
 u'-t': u'^(.*[jrwHiyEe926auOo\xea\xfb\xe2\xf4])$',
 u'-z': u'^r6mi$',
 u'9r-2z': u'^(.*[bvrE96O][rw6uOo\xf4][td])9r$',
 u'9r-ris': u'^prOtEkt9r$',
 u'e-Er': u'^(.*[mnJjlrwHiyEe926auOo\xea\xfb\xe2\xf4][ptkbdgfsSvzZmnJNjlriEe6a\xea\xe2][Zj])e$',
 u'f-v': u'^(.*[ptkbdgfsSvzZmnJNjrwHE96aO\xea\xfb\xe2\xf4][iyEe926uOo])f$',
 u'k-S': u'^sEk$',
 u'o-El': u'^(.*[ptbdfsvz])o$',
 u'o-Ot': u'^(.*[fsvzj])o$',
 u'u-Ol': u'^fu$',
 u'\xe2-an': u'^rOm\xe2$',
 u'\xea-En': u'^(.*[ptkbdgfsSvzZmnJNjlrE6\xea])\xea$',
 u'\xea-in': u'^(.*[ptkbdgfsSvzZmnJNlr])\xea$',
 u'\xf4-On': u'^(.*[pkbgfvmJNr])\xf4$',
 u'\xfb-yn': u'^(.*[bE96O][mr])\xfb$'}

In [321]:
rePatron=r1.patrons["e-Er"]
print rePatron
m=re.match(rePatron,"pr6mje")
if m:
    print m.group()

^(.*[mnJjlrwHiyEe926auOoêûâô][ptkbdgfsSvzZmnJNjlriEe6aêâ][Zj])e$
pr6mje


In [322]:
matchPatron=paradigme.ms.str.contains(rePatron,regex=True,na=False)

/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


In [324]:
pdMatch=paradigme[matchPatron]
positiveExtension=pdMatch[pdMatch["ms"].replace("e$","Er", regex=True)==pdMatch["fs"]][["ms"]]
overExtension=pdMatch[pdMatch["ms"].replace("e$","Er", regex=True)!=pdMatch["fs"]][["ms"]]
reliability(rePatron,positiveExtension["ms"],pdMatch["ms"])

17 23 0.739130434783


In [364]:
def regexPositions(regex):
    m=re.match(ur"^\^\((.*)\)(.*)\$$",regex)
    if m:
        context=[e for e in re.split(ur"\[|\]",m.group(1)) if e!=""]
        fix=list(m.group(2))
        return context+fix
    else:
        return list(regex)

In [393]:
positions=regexPositions(rePatron)
print rePatron
gReliability=reliability(rePatron,pdMatch["ms"],paradigmes["ms"])
variations=[[] for v in range(len(positions))]
for numPosition,regex in enumerate(positions):
    if regex!=".*" and len(regex)>1:
        principal="".join(fs.lattice[regex].extent)
        variations[numPosition].append(principal)
        for neighbor in fs.lattice[regex].lower_neighbors:
            lowNeighbor="".join(neighbor.extent)
            variations[numPosition].append(lowNeighbor)
    else:
        variations[numPosition].append(regex)

^(.*[mnJjlrwHiyEe926auOoêûâô][ptkbdgfsSvzZmnJNjlriEe6aêâ][Zj])e$


In [ ]:
vReliabilities={}
for variation in it.product(*variations):
    vReliability=reliability(patron2regexp(variation),pdMatch["ms"],paradigmes["ms"])
    if vReliability>=gReliability:
        print "{0:.4f}".format(vReliability),variation
        vReliabilities[variation]=vReliability
vReliabilities

In [527]:
def comparerPosition(p1,p2):
    if p1==".*":
        if p2==".*":
            return "="
        else:
            return ">"
    elif p2==".*":
        return "<"
    else:
        union=set(p1)|set(p2)
        if set(p1)==set(p2):
            return "="
        elif set(p1)==union:
            return ">"
        elif set(p2)==union:
            return "<"
        else:
            return u"≠"

def definirAlignement(patron):
    longueur=len(patron)
    jokers=(0,0)
    if patron[0]==".*" and patron[-1]==".*":
        print "patron, .*(___).*, alignement global"
        longueur=len(patron)-2
        jokers=(1,1)
    elif patron[0]==".*":
        print "patron, .*(___), alignement à droite"
        longueur=len(patron)-1
        jokers=(1,0)
    elif patron[-1]==".*":
        print "patron, (___).*, alignement à gauche"
        longueur=len(patron)-1
        jokers=(0,1)
    return (longueur, jokers)

def definirComparaison(comparaison):
    if u"≠" in comparaison:
        return u"≠"
    elif all([e=="=" for e in comparaison]):
        return u"="
    elif all([e=="=" or e=="<" for e in comparaison]):
        return "<"
    elif all([e=="=" or e==">" for e in comparaison]):
        return ">"
    else:
        return u"≠"

def comparerPatrons(p1,p2):
    if p1==p2: return "="
    (longueurP1,jokersP1)=definirAlignement(p1)   
    (longueurP2,jokersP2)=definirAlignement(p2)
    jokers=(jokersP1[0]+jokersP2[0],jokersP1[1]+jokersP2[1])
    if len(p1)!=len(p2):
        if len(p1)<len(p2):
            if jokersP1==(1,0):
                return comparerPatrons(p1,p2[len(p2)-len(p1):])
            elif jokersP1==(0,1):
                return comparerPatrons(p1,p2[:len(p2)-len(p1)+1])
            elif jokersP1==(1,1):
                print "pas d'alignement"
                return u"X"
        elif len(p2)<len(p1):
            if jokersP2==(1,0):
                return comparerPatrons(p1[len(p1)-len(p2):],p2)
            elif jokersP2==(0,1):
                return comparerPatrons(p1[:len(p1)-len(p2)+1],p2)
            elif jokersP2==(1,1):
                print "pas d'alignement"
                return u"X"

        else:
            return None
    else:
        print jokers
        comparaison12=[]
        for l in range(len(p1)):
            comparaison12.append(comparerPosition(p1[l],p2[l]))
        print comparaison12
        return definirComparaison(comparaison12)
'''
        if jokers==(2,2):
            for 
            return u"≠"
        elif jokers==(0,0):
            comparaison12=[]
            print longueurP1
            for l in range(longueurP1):
                comparaison12.append(comparerPosition(p1[l],p2[l]))
            return definirComparaison(comparaison12)
        elif jokers==(1,1)
'''            
    
class ReliabilityLattice():
    ''''''
    def __init__(self):
        self.lattice={}
        
    def addReliability(patron,reliability):
        for element in self.lattice:
            print element

In [530]:
print comparerPatrons([u"ai",u"bdg"],[".*",u"bdgptk"])

patron, .*(___), alignement à droite
(1, 0)
['<', '<']
<


In [198]:
setP1=u"ptkbdgfsSvzZjlrwHiyEe926uOo"
#setP1=u"jrwHiyEe926auOoêûâô"
P1= fs.lattice[setP1].extent
print fs.lattice[setP1].intent

(u'-low', u'-nas')


In [199]:
setI1="".join(overExtension["ms"].str[-1].unique().tolist())
I1= fs.lattice[setI1].extent
print fs.lattice[setI1].intent

(u'+son', u'-ant', u'-cor', u'-lat', u'+cont', u'+voice')


In [207]:
intersect="".join(list(set(P1) & set(I1)))
intruders="".join(list(fs.lattice[intersect].extent))
nbIntrudersTotal=len(intruders)
print "nbIntrudersTotal", nbIntrudersTotal

nbIntrudersTotal 14


In [209]:
lowerNeighbors=fs.lattice[setP1].lower_neighbors
for neighbor in lowerNeighbors:
    goodNeighbor=True
    nbIntruders=0
    for intruder in intruders:
        if intruder in neighbor.extent:
            goodNeighbor=False
            nbIntruders+=1
    if goodNeighbor:
        print "no intruders"
        print neighbor.intent
        print "".join(neighbor.extent)            
    elif nbIntruders<nbIntrudersTotal:
        print "%d intruders"%nbIntruders
        print neighbor.intent
        print "".join(neighbor.extent)
        

8 intruders
(u'-low', u'-ATR', u'-nas')
ptkbdgfsSvzZjlrwHE96O
6 intruders
(u'-low', u'-round', u'-nas')
ptkbdgfsSvzZjlriEe6
8 intruders
(u'-back', u'-low', u'-nas')
ptbdfsvzjlHiyEe92
8 intruders
(u'-high', u'-low', u'-nas')
ptbdfsvzlrEe926Oo


In [214]:
overLen=overExtension["ms"].str.len().min()

In [215]:
def getPhonemes(position,dfCol):
    setPosition="".join(dfCol.str[-position].unique().tolist())

def getIntruders():
    

IndentationError: expected an indented block (<ipython-input-215-9541159eba54>, line 5)

In [287]:
type(pdMatch["ms"])
    

pandas.core.series.Series

In [378]:
def dropParentheses(regex):
    result=re.sub(ur"\((.*)\)",ur"\1",regex)
    if "(" in result or ")" in result:
        print "pb de parenthèses"
    else:
        return result

def reliability(regex,dfPositives,dfTargets):
    regex=dropParentheses(regex)
    positives=dfPositives.dropna().reindex()
    targets=dfTargets.dropna().reindex()
    nbTargets=targets[targets.str.match(regex)].count()
    nbPositives=positives[positives.str.match(regex)].count()
    result=float(nbPositives)/nbTargets
    if debug: print nbPositives,nbTargets,result
    return result

In [314]:
reliability(rePatron,pdMatch["ms"],paradigmes["ms"])

353 904 0.390486725664


In [315]:
print dropParentheses(ur"^(test[^a])[^)].*")

pb de parenthèses
None
